In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('../CSV/19.01.2026/fixtures.csv')

# Convert date columns to datetime format
df['created_at'] = pd.to_datetime(df['created_at'])

print(df.head())
print(df.info())

                                     id                          game_week_id  \
0  00c2569b-cc74-4eca-9b0c-3d03feff2fcf  00c76044-16a0-4acf-a0a5-8de00e9cbc06   
1  03ee0cec-342e-4c89-8590-a70dfaffc42e  3fbb2bf0-cef3-48b2-9d04-ecba8bd4165b   
2  03f17591-9c55-4c93-8131-3ff2a3385c48  e1083685-268d-4376-bbc8-14fe74194d46   
3  0487bc30-fc05-4236-a6a5-eafcaded292b  37bc4415-6386-4c1c-9a8b-453802fa72e1   
4  058951f7-3051-4df0-99ba-a86590b9c725  e1083685-268d-4376-bbc8-14fe74194d46   

   fixture_number  home_team               away_team  home_score  away_score  \
0               3    Burnley                  Fulham         2.0         3.0   
1              10    Everton                West Ham         1.0         1.0   
2               5  Tottenham                 Burnley         3.0         0.0   
3               3    Arsenal  Brighton & Hove Albion         2.0         1.0   
4               9    Man Utd                 Arsenal         0.0         1.0   

                        created_